# 気象庁HTMLテーブル解析 PoC

このノートブックでは、気象庁のサンプルHTML（例: `../tests/resources/sample_jma.html`）を使い、
- pandas.read_html
- Beautiful Soup
の2通りで風況データテーブルを抽出する方法を比較します。

## 1. サンプルHTMLの準備

`../tests/resources/sample_jma.html` に気象庁のHTMLを保存してください。

In [13]:
# pandas.read_html を使ったテーブル抽出
import pandas as pd
from io import StringIO

html_path = "../resources/sample_jma.html"

# ファイルを明示的にutf-8で読み込む
with open(html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

# id="tablefix1" のテーブルのみをBeautifulSoupで抽出し、そのHTML断片をpandas.read_htmlに渡す
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, "lxml")
table = soup.find("table", id="tablefix1")
if table:
    table_html = str(table)
    try:
        dfs = pd.read_html(StringIO(table_html), flavor="lxml")
        print(f"検出されたテーブル数: {len(dfs)}")
        display(dfs[0])
    except Exception as e:
        print(f"pandas.read_htmlでの抽出に失敗: {e}")
else:
    print("id=tablefix1 のテーブルが見つかりませんでした")

検出されたテーブル数: 1


時分 気圧(hPa)         降水量 (mm) 気温 (℃) 相対湿度 (％) 風向・風速(m/s)                 \
        時分      現地      海面 降水量 (mm) 気温 (℃) 相対湿度 (％)         平均   風向 最大瞬間 風向.1   
0    00:10  1007.1  1009.9       --   17.6       87        1.6  北北東  3.4   北東   
1    00:20  1007.0  1009.8       --   17.9       84        1.2  北北東  2.6    北   
2    00:30  1007.0  1009.8       --   17.8       83        1.8  北北東  3.6  北北東   
3    00:40  1007.1  1009.9       --   18.3       78        2.4   北東  3.9  東北東   
4    00:50  1007.1  1009.9       --   17.6       82        1.9  北北東  3.6    北   
..     ...     ...     ...      ...    ...      ...        ...  ...  ...  ...   
139  23:20  1019.1  1022.0       --   16.9       74        1.4    東  3.8   北東   
140  23:30  1019.1  1022.0       --   17.3       71        3.0  東北東  5.3    東   
141  23:40  1018.9  1021.8       --   17.7       67        2.8  東南東  4.4  東南東   
142  23:50  1018.9  1021.8       --   17.2       70        2.1  東北東  4.2  東北東   
143  24:00  1018.8  1021.7       --   17.3       70        2.3  東北東  4.3  東北東   

    日照 時間 (分)  
    日照 時間 (分)  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
..        ...  
139       NaN  
140       NaN  
141       NaN  
142       NaN  
143       NaN  

[144 rows x 11 columns]

In [14]:
# Beautiful Soup を使ったテーブル抽出
from bs4 import BeautifulSoup

html_path = "../resources/sample_jma.html"

# ファイルを明示的にutf-8で読み込む
with open(html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, "lxml")

# 例: id="tablefix1" のテーブルを抽出
data_table = soup.find("table", id="tablefix1")
if data_table:
    rows = data_table.find_all("tr")
    for row in rows[2:]:  # ヘッダー行をスキップ
        cols = row.find_all("td")
        if len(cols) >= 3:
            time = cols[0].text.strip()
            direction = cols[1].text.strip()
            speed = cols[2].text.strip()
            print(f"{time}, {direction}, {speed}")
else:
    print("指定したテーブルが見つかりませんでした")

00:10, 1007.1, 1009.9
00:20, 1007.0, 1009.8
00:30, 1007.0, 1009.8
00:40, 1007.1, 1009.9
00:50, 1007.1, 1009.9
01:00, 1007.3, 1010.1
01:10, 1007.4, 1010.2
01:20, 1007.5, 1010.3
01:30, 1007.5, 1010.3
01:40, 1007.6, 1010.4
01:50, 1008.0, 1010.8
02:00, 1007.9, 1010.7
02:10, 1008.1, 1010.9
02:20, 1008.2, 1011.0
02:30, 1008.6, 1011.4
02:40, 1008.7, 1011.5
02:50, 1008.9, 1011.7
03:00, 1009.0, 1011.8
03:10, 1008.9, 1011.7
03:20, 1009.1, 1011.9
03:30, 1009.2, 1012.0
03:40, 1009.2, 1012.0
03:50, 1009.4, 1012.2
04:00, 1009.6, 1012.4
04:10, 1009.7, 1012.5
04:20, 1009.7, 1012.5
04:30, 1009.8, 1012.6
04:40, 1010.5, 1013.3
04:50, 1010.6, 1013.5
05:00, 1010.7, 1013.6
05:10, 1010.7, 1013.6
05:20, 1010.9, 1013.8
05:30, 1011.1, 1014.0
05:40, 1011.3, 1014.2
05:50, 1011.3, 1014.2
06:00, 1011.6, 1014.5
06:10, 1011.9, 1014.8
06:20, 1011.9, 1014.8
06:30, 1012.1, 1015.0
06:40, 1012.5, 1015.4
06:50, 1012.6, 1015.5
07:00, 1012.6, 1015.5
07:10, 1012.7, 1015.6
07:20, 1012.7, 1015.6
07:30, 1012.7, 1015.6
07:40, 101

## 2. 比較ポイント

- コード量・実装の簡単さ
- テーブル構造の変化への耐性
- 必要なデータの抽出容易性

この結果をもとに、どちらのライブラリを採用するか決定します。